In [1]:
import os
from PIL import Image
from datetime import datetime, timedelta
import random
import csv

In [7]:
INPUT_DIR = 'F:\\USFWS_Cameras\\Data\\LANWR\\LANWR_Images_Nov2017\\animal\\Buena Vista Guzzler'
OUTPUT_DIR = 'F:\\USFWS_Cameras\\working\\ZooniverseUploads\\LANWR_Guzzlers\\BuenaVista'
TIME_THRESHOLD = timedelta(minutes=2)

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [8]:
image_files = [file for file in os.listdir(INPUT_DIR) if file.lower().endswith(('.jpg', '.jpeg', '.png'))]

In [9]:
def get_image_time(image_path):
    img = Image.open(image_path)
    exif_data = img._getexif()
    capture_time = exif_data.get(36867)  # EXIF code for capture time
    return datetime.strptime(capture_time, '%Y:%m:%d %H:%M:%S')

def copy_image(source_path, target_path):
    img = Image.open(source_path)
    img.save(target_path)

In [10]:
sets = []
current_set = [image_files[0]]

for i in range(1, len(image_files)):
    prev_image = current_set[-1]
    current_image = image_files[i]
    
    prev_time = get_image_time(os.path.join(INPUT_DIR, prev_image))
    current_time = get_image_time(os.path.join(INPUT_DIR, current_image))
    
    time_difference = current_time - prev_time
    
    if time_difference <= TIME_THRESHOLD:
        current_set.append(current_image)
    else:
        sets.append(current_set)
        current_set = [current_image]

# Add the last set
sets.append(current_set)

# Copy random image from each set to output directory
summary = []

for set_num, image_set in enumerate(sets):
    random_image = random.choice(image_set)
    source_path = os.path.join(INPUT_DIR, random_image)
    target_path = os.path.join(OUTPUT_DIR, f'{random_image}.jpg')
    copy_image(source_path, target_path)
    summary.append({'set_num': set_num + 1, 'images': image_set, 'selected_image': random_image})

# Write summary to CSV
with open('bunavista_summary.csv', 'w', newline='') as csvfile:
    fieldnames = ['set_num', 'images', 'selected_image']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(summary)

In [6]:
import os
from PIL import Image
from datetime import datetime, timedelta
import random
import csv

INPUT_DIR = 'F:\\USFWS_Cameras\\Data\\LANWR\\LANWR_Images_Nov2017\\animal\\Buena Vista Guzzler'
TIME_THRESHOLD = timedelta(minutes=2)

def get_image_time(image_path):
    img = Image.open(image_path)
    exif_data = img._getexif()
    capture_time = exif_data.get(36867)  # EXIF code for capture time
    return datetime.strptime(capture_time, '%Y:%m:%d %H:%M:%S')

image_files = [file for file in os.listdir(INPUT_DIR) if file.lower().endswith(('.jpg', '.jpeg', '.png'))]

sets = []
current_set = [image_files[0]]

for i in range(1, len(image_files)):
    prev_image = current_set[-1]
    current_image = image_files[i]
    
    prev_time = get_image_time(os.path.join(INPUT_DIR, prev_image))
    current_time = get_image_time(os.path.join(INPUT_DIR, current_image))
    
    time_difference = current_time - prev_time
    
    if time_difference <= TIME_THRESHOLD:
        current_set.append(current_image)
    else:
        sets.append(current_set)
        current_set = [current_image]

sets.append(current_set)

# Create output directory
output_dir = os.path.join(INPUT_DIR, 'selected_images')
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Write summary to CSV
summary = []

for set_num, image_set in enumerate(sets):
    random_image = random.choice(image_set)
    for image in image_set:
        source_path = os.path.join(INPUT_DIR, image)
        target_path = os.path.join(output_dir, image)
        copy_image(source_path, target_path)
        selected = "Yes" if image == random_image else "No"
        summary.append({'set_num': set_num + 1, 'image': image, 'selected': selected})

# Write summary to CSV
input_dir_name = os.path.basename(os.path.normpath(INPUT_DIR))
summary_csv_path = os.path.join(INPUT_DIR, f'{input_dir_name}_summary.csv')

with open(summary_csv_path, 'w', newline='') as csvfile:
    fieldnames = ['set_num', 'image', 'selected']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(summary)


KeyboardInterrupt: 